# Asssignment 3

In [1]:
import spotipy
import json

from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [2]:
packages = {
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0"
}

In [3]:
load_dotenv()
spark = SparkSession.builder.appName('assignment3').config("spark.jars.packages", ",".join(packages)).getOrCreate()
producer = KafkaProducer(bootstrap_servers='localhost:9092')
spark.sparkContext.setLogLevel("ERROR")

23/05/26 12:15:21 WARN Utils: Your hostname, HP-Elite-845 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlan0)
23/05/26 12:15:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hajta2/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hajta2/.ivy2/cache
The jars for the packages stored in: /home/hajta2/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a546229c-b069-4442-8f69-9397669a121f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 582ms :: artifacts dl 19ms
	:: 

In [4]:
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [5]:
producer.send('tracks_topic', b'')
producer.send('audio_features_topic', b'')
producer.send('trending_tracks_topic', b'')
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "tracks_topic, audio_features_topic, trending_tracks_topic")
    .option("startingOffsets", "latest")
    .load()
)

In [6]:
tracks_stream = df.selectExpr("CAST(value AS STRING)").filter("topic = 'tracks_topic'")
features_stream = df.selectExpr("CAST(value AS STRING)").filter("topic = 'audio_features_topic'")
trending_tracks_stream = df.selectExpr("CAST(value AS STRING)").filter("topic = 'trending_tracks_topic'")

In [7]:
tracks_query = tracks_stream.writeStream.format("memory").queryName("tracks").start()
features_query = features_stream.writeStream.format("memory").queryName("features").start()
trending_tracks_query = trending_tracks_stream.writeStream.format("memory").queryName("trending_tracks").start()

In [8]:
saved_tracks = sp.current_user_saved_tracks(limit=50)
for item in saved_tracks['items']:
    track = item['track']
    producer.send('tracks_topic', json.dumps(track).encode('utf-8'))
    audio_features = sp.audio_features(track['id'])
    producer.send('audio_features_topic', json.dumps(audio_features[0]).encode('utf-8'))

In [9]:
trending_playlist_id = sp.featured_playlists(limit=1, country='HU', locale='hu_HU')['playlists']['items'][0]['id']
trending_tracks = sp.playlist_tracks(trending_playlist_id, limit=50, offset=0, market='HU')['items']
for item in trending_tracks:
    track = item['track']
    producer.send('trending_tracks_topic', json.dumps(track).encode('utf-8'))
    audio_features = sp.audio_features(track['id'])
    producer.send('audio_features_topic', json.dumps(audio_features[0]).encode('utf-8'))

In [10]:
tracks = spark.sql("select * from tracks")
features = spark.sql("select * from features")
trending_tracks = spark.sql("select * from trending_tracks")

In [11]:
print(f"Tracks: {tracks.count()}")
print(f"Features: {features.count()}")
print(f"Trending tracks: {trending_tracks.count()}")

Tracks: 50
Features: 99
Trending tracks: 50


In [12]:
tracks_query.stop()
features_query.stop()
trending_tracks_query.stop()

23/05/26 12:15:38 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 34, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@6f08c9d0] is aborting.
23/05/26 12:15:38 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 34, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@6f08c9d0] aborted.
23/05/26 12:15:39 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 35, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@2bc92aaa] is aborting.
23/05/26 12:15:39 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 35, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@2bc92aaa] aborted.


In [13]:
tracks.printSchema()
features.printSchema()
trending_tracks.printSchema()

root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)



23/05/26 12:15:39 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:267)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.$anonfun$run$1(WriteToDataSourceV2Exec.scala:464)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1563)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:509)
	at org.apache.spark.sql.execution.datasources.v2

In [14]:
tracks_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("artists", StringType(), True),
        StructField("duration_ms", IntegerType(), True),
        StructField("popularity", FloatType(), True),
    ]
)

tracks_parsed = tracks.withColumn(
    "parsed_value", from_json(col("value"), tracks_schema)
).select("parsed_value.*")
trending_tracks_parsed = trending_tracks.withColumn(
    "parsed_value", from_json(col("value"), tracks_schema)
).select("parsed_value.*")

In [15]:
tracks_parsed.printSchema()
trending_tracks_parsed.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- popularity: float (nullable = true)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- popularity: float (nullable = true)



In [16]:
print(f"Tracks: {tracks_parsed.count()}")
print(f"Trending tracks: {trending_tracks_parsed.count()}")

Tracks: 50
Trending tracks: 50


In [17]:
print(f"Tracks: {tracks_parsed.show(1)}")
print(f"Trending tracks: {trending_tracks_parsed.show(1)}")

+--------------------+-----+--------------------+-----------+----------+
|                  id| name|             artists|duration_ms|popularity|
+--------------------+-----+--------------------+-----------+----------+
|0pAiyIHt9DyHOjWgF...|Steal|[{"external_urls"...|     219945|      68.0|
+--------------------+-----+--------------------+-----------+----------+
only showing top 1 row

Tracks: None
+--------------------+--------+--------------------+-----------+----------+
|                  id|    name|             artists|duration_ms|popularity|
+--------------------+--------+--------------------+-----------+----------+
|7n5JBAnjVBTFgTEsd...|Inkasszó|[{"external_urls"...|     149673|      38.0|
+--------------------+--------+--------------------+-----------+----------+
only showing top 1 row

Trending tracks: None


In [18]:
features_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("danceability", FloatType(), True),
        StructField("energy", FloatType(), True),
        StructField("key", IntegerType(), True),
        StructField("loudness", FloatType(), True),
        StructField("mode", IntegerType(), True),
        StructField("speechiness", FloatType(), True),
        StructField("acousticness", FloatType(), True),
        StructField("instrumentalness", FloatType(), True),
        StructField("liveness", FloatType(), True),
        StructField("valence", FloatType(), True),
        StructField("tempo", FloatType(), True),
    ]
)

features_parsed = features.withColumn(
    "parsed_value", from_json(col("value"), features_schema)
).select("parsed_value.*")

In [19]:
features_parsed.printSchema()

root
 |-- id: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)



In [20]:
print(f"Features: {features_parsed.count()}")

Features: 99


In [21]:
print(f"Features: {features_parsed.show(1)}")

+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+
|                  id|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|
+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+
|0pAiyIHt9DyHOjWgF...|       0.729| 0.456|  6|  -6.795|   0|     0.0528|        0.62|         1.33E-4|  0.0919|  0.221|110.999|
+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+
only showing top 1 row

Features: None


In [22]:
joined_tracks = tracks_parsed.join(features_parsed, tracks_parsed.id == features_parsed.id).drop(features_parsed.id)
joined_trending_tracks = trending_tracks_parsed.join(features_parsed, trending_tracks_parsed.id == features_parsed.id).drop(features_parsed.id)

In [23]:
print(f"Joined tracks: {joined_tracks.count()}")
print(f"Joined trending tracks: {joined_trending_tracks.count()}")

Joined tracks: 50
Joined trending tracks: 49


In [24]:
joined_tracks.printSchema()
joined_trending_tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- popularity: float (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- popularity: float (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode

In [25]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo"
], outputCol="features")
assembled_tracks = assembler.transform(joined_tracks)

In [26]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(assembled_tracks)
scaled_tracks = scalerModel.transform(assembled_tracks)

In [27]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(predictionCol="prediction", featuresCol="scaledFeatures", metricName="silhouette", distanceMeasure="squaredEuclidean")
kmeans = KMeans(featuresCol="scaledFeatures", predictionCol="prediction", k=3, seed=1)
model = kmeans.fit(scaled_tracks)
predictions = model.transform(scaled_tracks)